# Google Drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/Othercomputers/Il mio computer/Tesi/Computing/Datasets')

# Initialization

In [54]:
!pip install -r requirements.txt

You should consider upgrading via the 'C:\Users\mirco\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.

In [1]:
from ipywidgets import widgets, interact, interact_manual
from IPython.display import display
from halo import HaloNotebook as Halo
from tqdm.notebook import tqdm
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from skopt import gp_minimize
from random import randint
from itertools import product
from datetime import datetime
from time import sleep
from libraries.iNMF_wrapper import inmf
from libraries.parameters_widgets import threaded
import libraries.pca
import torch
import numpy as np
import os
import contextlib
import io
import sys
import math
import pandas
import sklearn.metrics
import warnings
warnings.filterwarnings('ignore')

## Threaded execution
Using modin and Dask slightly increases performances, but requires more RAM.

In [2]:
display(threaded)

Checkbox(value=False, description='Threaded DataFrame manipulation')

In [3]:
if threaded.value:
    import modin.pandas as pd
    import dask
    from dask.distributed import Client

    # dask.config.set(temporary_directory='C:\DaskTemp')
    client = Client(n_workers=4, threads_per_worker=2)  # More workers = more RAM needed

else:
    import pandas as pd

## Paths 

In [4]:
datasets_root = '../../Datasets/9_normalized'
labels_path = '../../Datasets/6_downcasted'
results_root = '../Results'
output_root = '../../Datasets/output'

## Functions

In [5]:
def pca(datasets_dict, threshold, ratio=None, n_components=None, log=False):
    assert(ratio or n_components)

    for name, dataset in (bar := tqdm(datasets_dict.items(), leave=False)):

        if len(dataset.columns) > threshold:
            bar.set_description(f'Computing PCA on {name}')
            pca_results = compute_pca(dataset)
            bar.set_description('Done')

            if ratio is not None:
                selected_components = cumulative_ratio(pca_results, ratio)
            else:
                selected_components = select_components(pca_results, n_components)

            if log:
                bar.write(f'{name}: selected {len(selected_components)}/{len(dataset.columns)} features ({len(selected_components)/len(dataset.columns)*100:.2f}%)')
        else:
            if log:
                bar.write(f'{name}: smaller than features threshold')
            selected_components = [(col, 0) for col in dataset.columns]

        bar.set_description('Done')

        datasets_dict[name] = dataset[[component[0] for component in selected_components]]

    return datasets_dict

In [6]:
# Given the dataset name and its extension, returns its name without the '.\<extension>'
def dataset_name(dataset_file: str, extension: str) -> str:
    return dataset_file.replace(f'.{extension}', '')

In [7]:
# Given a dataset and, optionally, the number of components; computes PCA and returns a list of pairs, where each pair is: <feature name>, <explained_variance_ratio>
def compute_pca(dataset, n_components=None):
    pca = PCA(n_components=n_components)
    pca.fit(dataset)
    pca_results = [(pca.feature_names_in_[i], pca.explained_variance_ratio_[i]) for i in range(len(pca.components_))]

    return pca_results

In [8]:
def select_components(pca_results, n_components):
    selected_components = pca_results[:n_components]

    return selected_components

In [9]:
# Selects features to reach <ratio>
def cumulative_ratio(pca_results, ratio):
    selected_components = []

    i = 0

    while sum([component_ratio[1] for component_ratio in selected_components]) <= ratio:
        selected_components.append(pca_results[i])
        i += 1

    return selected_components

In [10]:
# True: CUDA libraries availables
def gpu_available() -> bool:
    return torch.cuda.is_available()

In [11]:
# Loads dataset from file. If bar is not None, updates its' description
def load_dataset(dataset_file, root, bar: tqdm = None) -> pd.DataFrame:
    name = dataset_name(dataset_file, 'xz')
    path = os.path.join(root, dataset_file)
    if bar:
        bar.set_description(f'Loading "{name}" from "{path}"')
    dataset = pd.read_pickle(path)

    return dataset

In [12]:
def get_parameters() -> dict:
    parameters = {'tol': tol.value,
                  'lam': lam.value,
                  'batch_max_iter': batch_max_iter.value,
                  'n_components': n_components.value}

    if use_gpu.value:
        parameters['use_gpu'] = use_gpu.value
    else:
        parameters['n_jobs'] = n_threads.value

    return parameters

In [13]:
# Call to iNMF helper
def call_inmf(datasets_list: list, components, tol, lam, batch_max_iter) -> dict:
    parameters = {'tol': tol,
                  'lam': lam,
                  'batch_max_iter': batch_max_iter,
                  'n_components': components}

    if use_gpu.value:
        parameters['use_gpu'] = True
    else:
        parameters['n_jobs'] = n_threads.value

    matrices = [dataset.transpose().to_numpy() for dataset in datasets_list]

    with contextlib.redirect_stdout(None):
        H, W, V, err = inmf(X=matrices, **parameters)

    return {'H': H, 'W': W, 'V': V, 'err': err}

In [14]:
# Counts decimals positions of the given value
def count_decimals(number) -> int:
    if isinstance(number, int):
        return 0

    string = str(number)

    if '.' in string:
        decimals = string.split('.')[1]
        return len(decimals)
    else:
        return int(string.split('e-')[1])

In [15]:
# Applies Min-Max normalization to the passed pandas Series
def min_max_norm(column: pd.Series):
    col_min = min(column)
    col_max = max(column)

    col_range = col_max - col_min

    if col_range == 0:
        return column

    return column.apply(lambda x: (x - col_min)/col_range)

In [16]:
# Applies column-based Min-Max normalization to the passed pandas DataFrame
def df_min_max_norm(df: pd.DataFrame) -> pd.DataFrame:
    df_ = df.copy()
    for col_name in df_.columns:
        df_[col_name] = df_[col_name].apply(min_max_norm)

    return df_

In [17]:
# Returns a list containing all possible sub-lists of l
def sub_lists(l: list):
    lists = []
    for i in range(len(l) + 1):
        for j in range(i):
            if len(l[j: i]) > 1:
                lists.append(l[j: i])
    return lists

In [18]:
# Return a pandas index containing all patients
def get_patients(datasets: dict):
    first_df = list(datasets.values())[0]
    return first_df.index

In [19]:
def get_silhoutte(df, labels):
    silhoutte = sklearn.metrics.silhouette_score(df, labels)

    return silhoutte

In [20]:
def W_to_df(W, patients):
    df = pandas.DataFrame(W).transpose()
    df.index = patients

    return df

In [21]:
def merge_pfi_dfi(df):
    df = df.merge(labels, how='inner', left_index=True, right_index=True)
    df['PFI'] = df['PFI'].astype(str)
    df['DFI'] = df['DFI'].astype(str)

    return df

In [22]:
def plot_W(W, patients, labels, color_column):
    fig = get_W_plot(W, patients, labels, color_column)
    fig.show()

In [23]:
def get_W_plot(W, patients, labels, color_column):
    W_df = W_to_df(W, patients)

    silhoutte = get_silhoutte(W_df, labels)

    W_df = merge_pfi_dfi(W_df)

    tridimensional = len(W_df.columns) - 2 == 3

    axes = {'x': 0, 'y': 1}
    fig_class = px.scatter

    if tridimensional:
        axes['z'] = 2
        fig_class = px.scatter_3d

    fig = fig_class(W_df, **axes, color=color_column, width=800, height=800,
                    title=f'{3 if tridimensional else 2} Components | Silhoutte: {silhoutte}')
    fig.update_traces(marker=dict(size=3))

    return fig

In [24]:
def plot_error_vs_silhoutte(results, parameter_name, title):
    parameter_header = f'{parameter_name.capitalize()}'
    df = pandas.DataFrame(results, columns=[parameter_header, 'Error',  'Silhoulette'])
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Error", "Silhoulette"))
    fig.append_trace(go.Scatter(x=df[parameter_header], y=df['Error']), row=1, col=1)
    fig.append_trace(go.Scatter(x=df[parameter_header], y=df['Silhoulette']), row=2, col=1)
    fig.update_layout(height=800, showlegend=False, title_text=title)
    fig.show()

In [25]:
def load_all_datasets(datasets_root) -> dict:
    datasets_filenames = os.listdir(datasets_root)
    if 'tcga_cdr_brca_labels.xz' in datasets_filenames:
        datasets_filenames.remove('tcga_cdr_brca_labels.xz')

    datasets = {}

    for dataset in (bar := tqdm(datasets_filenames)):
        datasets[dataset_name(dataset, 'xz')] = load_dataset(dataset, datasets_root, bar)

        bar.set_description('Datasets loaded')

    print(f'{len(datasets)} datasets loaded:', '\n'.join(datasets.keys()), sep='\n')

    return datasets

### Labels loading

In [26]:
with Halo(text=f'Loading labels dataset...', spinner='dots'):
    labels = load_dataset('tcga_cdr_brca_labels.xz', labels_path)

if not isinstance(labels, pandas.DataFrame):
    labels = labels._to_pandas()

print('Loaded')

Output()

Loaded


## Fixed parameters

In [27]:
# Convergence tolerance
tol = 1e-4

# Regularization parameter
lam = 1

batch_max_iter = 200

## Parameters

In [28]:
datasets_names = [dataset_name(filename, 'xz') for filename in os.listdir(datasets_root)]

datasets_lists = sub_lists(list(datasets_names))
n_components = [2, 3, 100, 200]
pca_param = [0.75, 0.9, 100, 200]  # <=1: cumulative ratio, >1: max selected components

parameters = {'datasets': datasets_lists,
              'n_components': n_components,
              'pca': pca_param}

## Generate tests

In [29]:
completed_tests = widgets.BoundedIntText(value=0, min=0, max=9999999999999, description='Completed tests:')
display(completed_tests)

BoundedIntText(value=0, description='Completed tests:', max=9999999999999)

In [30]:
# Generate all combinations of parameters
prod = list(product(*list(parameters.values())))

# Build a category: values dictionary for each test
tests = [{key: p[i] for i, key in enumerate(parameters.keys())} for p in prod]

# Remove already completed tests
tests = tests[completed_tests.value:]

tests = [test for test in tests if test['n_components'] != 200]

tests = [{'datasets': ['miRNA_mor', 'miRNA_vst', 'mRNA_mor', 'mRNA_vst'],
          'n_components': 100,
          'pca': 100}]

print(f'{len(tests)} tests')

1 tests


## Hardware parameters

In [31]:
int64_info = np.iinfo(np.int64)
max_int = int64_info.max

# Threads number
n_threads = widgets.IntSlider(value=-1, min=-1, max=64, step=1, description='CPU threads:', continuous_update=False)

# Use GPU
use_gpu = widgets.Checkbox(value=False, description='Use GPU')

In [32]:
# Displays hardware related parameters
def display_hw_parameters():
    if gpu_available():
        display(use_gpu)
        display(n_threads)
    else:
        display(n_threads)

In [33]:
display_hw_parameters()

Checkbox(value=False, description='Use GPU')

IntSlider(value=-1, continuous_update=False, description='CPU threads:', max=64, min=-1)

## Datasets loading and normalization

In [34]:
datasets = load_all_datasets(datasets_root)

for name, dataset in (bar := tqdm(datasets.items())):
    bar.set_description(f'Normalizing {name}...')
    #datasets[name] = dataset.apply(min_max_norm)
    bar.set_description('Normalization done')

  0%|          | 0/7 [00:00<?, ?it/s]

7 datasets loaded:
clinical_data
cnv.score
met_Mval
miRNA_mor
miRNA_vst
mRNA_mor
mRNA_vst


  0%|          | 0/7 [00:00<?, ?it/s]

# Parametric iNMF

### PCA phase

In [35]:
def pca_buffer_key(test):
    return f"{test['pca']}-{test['datasets']}"

In [36]:
def pca_phase(pca_buffer: dict, test):
    pca_param = test['pca']
    if pca_buffer_key(test) not in pca_buffer:
        if pca_param <= 1:
            pca_buffer[pca_buffer_key(test)] = pca(datasets.copy(), threshold=100, ratio=pca_param)

        else:
            pca_buffer[pca_buffer_key(test)] = pca(datasets.copy(), threshold=100, n_components=pca_param)

### Report generation

In [37]:
def generate_report(test, err, silhoutte):
    report = test
    report['error'] = err
    report['silhoutte'] = silhoutte

    return report

### Image to file

In [38]:
def create_result_dir(now):
    dir_name = now.strftime("%d-%m-%Y_%H-%M-%S")

    path = os.path.join(results_root, dir_name)
    if os.path.isdir(path):
        alt = 0
        while os.path.isdir(os.path.join(results_root, f'{dir_name}_{alt}')):
            alt += 1

        path = os.path.join(results_root, f'{dir_name}_{alt}')

    os.mkdir(path)

    return path

In [39]:
def generate_description(result):
    description = '\n'.join([f'{cat.capitalize()}: {values}' for cat, values in result.items()])

    return description

In [40]:
def save_fig(fig, result):
    description = generate_description(result)

    now = datetime.now()
    result_dir = create_result_dir(now)
    fig_name = f'{now.strftime("%d-%m-%Y_%H-%M-%S")}.png'
    desc_name = f'{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'

    fig_path = os.path.join(results_root, result_dir, fig_name)
    desc_path = os.path.join(results_root, result_dir, desc_name)

    with open(desc_path, 'w') as file:
        file.write(description)

    fig.write_image(fig_path)

### Results to csv

In [41]:
def save_results(results, name):
    path = os.path.join(results_root, name)
    results.to_csv(path, index=False)

In [44]:
results_columns = list(parameters.keys()) + ['error', 'silhoutte']
results = pd.DataFrame(columns=results_columns)

now = datetime.now()
csv_name = f'{now.strftime("%d-%m-%Y_%H-%M-%S")}.csv'

pca_datasets = {}

for test in (tests_bar := tqdm(tests)):
    tests_bar.set_postfix_str(f'{test}')

    tests_bar.set_description('PCA phase')
    pca_phase(pca_datasets, test)
    tests_bar.set_description('PCA completed')

    tests_bar.set_description('Computing iNMF')
    pca_datasets_subset = {name: dataset for name, dataset in pca_datasets[pca_buffer_key(test)].items() if name in test['datasets']}

    _, W, _, err = inmf(datasets=pca_datasets_subset.values(), n_components=test['n_components'], tol=tol, lam=lam, batch_max_iter=batch_max_iter)
    silhoutte = get_silhoutte(W_to_df(W, get_patients(pca_datasets[pca_buffer_key(test)])), labels['PFI'])

    report = generate_report(test, err, silhoutte)
    results = results.append(report, ignore_index=True)

    if 2 <= test['n_components'] <= 3:
        tests_bar.set_description('Saving image')
        fig = get_W_plot(W, get_patients(datasets), labels['PFI'], color_column='PFI')
        save_fig(fig, report)

    tests_bar.set_description('Saving results')
    save_results(results, csv_name)

    tests_bar.set_description('Cooling down CPU')
    # sleep(5)  # Useful when computing iNMF on CPU

    tests_bar.set_description('Done')
    tests_bar.set_postfix('')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [42]:
test = tests[0]

In [43]:
_, W, _, err = inmf(datasets=pca_datasets_subset, n_components=3, tol=tol, lam=float(lam), batch_max_iter=batch_max_iter, n_threads=-1, use_gpu=True, algo='halsvar')

NameError: name 'pca_datasets_subset' is not defined